<a href="https://colab.research.google.com/github/jdasam/aat3020/blob/main/notebooks/1_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import string


In [2]:
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"


--2024-03-14 06:00:46--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439742 (429K) [text/plain]
Saving to: ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’

J. K. Rowling - Har 100%[===================>] 429.44K  --.-KB/s    in 0.04s   

2024-03-14 06:00:47 (11.4 MB/s) - ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’ saved [439742/439742]



In [4]:
def remove_punctuation(x):
  return x.translate(''.maketrans('', '', string.punctuation))

def make_tokenized_corpus(corpus):
  out= [ [y.lower() for y in remove_punctuation(sentence).split(' ') if y] for sentence in corpus]
  return [x for x in out if x!=[]]


In [5]:
with open("J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt", 'r') as f:
  strings = f.readlines()
sample_text = "".join(strings).replace('\n', ' ').replace('Mr.', 'mr').replace('Mrs.', 'mrs').split('. ')


In [8]:
corpus = make_tokenized_corpus(sample_text)
print(corpus[1]) # Corpus is a list of list of strings (words)

['they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense']


In [10]:
from copy import copy
copy(corpus[1])

['they',
 'were',
 'the',
 'last',
 'people',
 'youd',
 'expect',
 'to',
 'be',
 'involved',
 'in',
 'anything',
 'strange',
 'or',
 'mysterious',
 'because',
 'they',
 'just',
 'didnt',
 'hold',
 'with',
 'such',
 'nonsense']

In [11]:
from tqdm import tqdm

sample_sentence = ['they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense']

word_idx = 7
window_size = 2
selected_center_word = sample_sentence[word_idx]

word_pairs = []

for sample_sentence in tqdm(corpus): # tqdm shows the progress bar of for loop
  for word_idx, word in enumerate(sample_sentence):
    for context_idx in range(-window_size, window_size+1):
      if context_idx == 0: continue
      context_word_idx = word_idx + context_idx
      if context_word_idx >= len(sample_sentence) or context_word_idx < 0: continue
      # print(context_idx, context_word_idx, sample_sentence[word_idx], sample_sentence[context_word_idx])
      word_pair = [word, sample_sentence[context_word_idx]]
      word_pairs.append(word_pair)
len(word_pairs)

100%|██████████| 4682/4682 [00:00<00:00, 6817.90it/s]


282372

In [12]:
word_pairs[1000]

['mixing', 'dudley']

In [ ]:
# we have to make vocabulary

total_words = []
for sentence in corpus:
  # print(sentence)
  for word in sentence:
    total_words.append(word)

print(total_words[:100])
# for a in corpus:
#   for b in a:
#     c.append(b)
# always use proper variable name

In [ ]:
# we have to get the "unique" item among total words
vocab_set = set(total_words)
print(len(total_words), len(vocab_set))
print(vocab_set)

In [ ]:
# this will make error
# vocab_set[0] # set is not subscriptable because it has no order

vocab = list(vocab_set) # converting set into list, so that we can order the items
# sort the liset
vocab = sorted(vocab)
# vocab.sort()
print(vocab)


In [36]:
# how can we filter the vocab by its frequency?
from collections import Counter
word_counter = Counter(total_words)
# word_counter['hundreds']
len(word_counter)

# you can use word counter as dictionary
# In python dictionary, dict.keys() gives keys, and dict.values() give values,
# dict.items() give (key, value)
filtered_vocab = []
minimum_occur = 5
for key, value in word_counter.items():
  if value >= minimum_occur:
    filtered_vocab.append(key)
print(len(filtered_vocab))
filtered_vocab.sort() # you must not declare filtered_vocab = filtered_vocab.sort()

1701


In [ ]:
print(filtered_vocab)

In [46]:
# Filter the word_pairs using the vocab
# word_pairs, filtered_vocab
# word_pairs is a list of [word_a, word_b]

filtered_pairs = []
for pair in tqdm(word_pairs):
  word_a, word_b = pair
  # word_a = pair[0]
  # word_b = pair[1]
  # we will include this pair only if both words are in the vocab
  if word_a in filtered_vocab and word_b in filtered_vocab:
    filtered_pairs.append(pair)



 21%|██        | 59387/282372 [00:01<00:07, 31526.39it/s]


KeyboardInterrupt: 

In [47]:
# implement same algorithm with list comprehension
filtered_pairs = [pair for pair in word_pairs
                  if pair[0] in filtered_vocab and pair[1] in filtered_vocab]

In [48]:
len(filtered_pairs), len(word_pairs)

(233046, 282372)

In [52]:
filtered_pairs[1000]

['happily', 'she']

In [57]:
# convert word into index of vocab
filtered_vocab.index('happily')

index_pairs = []
for word_a, word_b in tqdm(filtered_pairs):
  word_a_idx = filtered_vocab.index(word_a)
  word_b_idx = filtered_vocab.index(word_b)
  index_pairs.append([word_a_idx, word_b_idx])
len(index_pairs)

100%|██████████| 233046/233046 [00:08<00:00, 28719.18it/s]


233046

In [58]:
filtered_vocab[100]

'behind'

In [61]:
# we can make it faster
# use dictionary to find the index of string

# adict['behind'] = 100
# tok2idx = {}
# for idx, word in enumerate(filtered_vocab):
#   tok2idx[word] = idx # word becomes key, idx becomes value

tok2idx = {word: idx for idx, word in enumerate(filtered_vocab)}

tok2idx['harry']

613

In [63]:
index_pairs = []
for word_a, word_b in tqdm(filtered_pairs):
  word_a_idx = tok2idx[word_a]
  word_b_idx = tok2idx[word_b]
  index_pairs.append([word_a_idx, word_b_idx])


100%|██████████| 233046/233046 [00:00<00:00, 537845.43it/s]


In [64]:
# Why we don't need idx2tok?

def convert_idx_pairs_to_string(pair):
  # pair: [int, int]
  idxa, idxb= pair
  return filtered_vocab[idxa], filtered_vocab[idxb]
convert_idx_pairs_to_string(index_pairs[0])

('harry', 'potter')

In [70]:
# we have to make random vectors for each word in the vocab
# we also have to decide the dimension of the vector

dim = 8
vocab_size = len(filtered_vocab)

word_vectors = torch.randn(vocab_size, dim) / 4
word_vectors, word_vectors.shape

(tensor([[ 0.2139,  0.2560, -0.1631,  ..., -0.5057, -0.0050, -0.1986],
         [-0.0261, -0.0950,  0.0102,  ..., -0.2876,  0.0060, -0.4206],
         [ 0.0215, -0.1089, -0.2158,  ..., -0.3356, -0.1273, -0.1058],
         ...,
         [-0.1933,  0.1937, -0.3518,  ...,  0.2688, -0.0190, -0.2909],
         [ 0.0930, -0.0169, -0.2924,  ..., -0.1347, -0.0863,  0.1162],
         [ 0.0539,  0.1383, -0.1870,  ...,  0.2358,  0.1057, -0.5139]]),
 torch.Size([1701, 8]))

In [79]:
# what is the vector for harry?
idx_of_harry = tok2idx['harry']
print(idx_of_harry)
vector_of_harry = word_vectors[idx_of_harry]
vector_of_harry

613


tensor([ 0.2799, -0.0560, -0.1426, -0.2151,  0.1446,  0.0032,  0.2464,  0.1845])

In [78]:
filtered_pairs[0], index_pairs[0]

(['harry', 'potter'], [613, 1085])

In [87]:
torch.set_printoptions(sci_mode=False)

print(vector_of_harry)
print(vector_of_potter)
print(vector_of_harry * vector_of_potter)
print(sum(vector_of_harry * vector_of_potter))

tensor([ 0.2799, -0.0560, -0.1426, -0.2151,  0.1446,  0.0032,  0.2464,  0.1845])
tensor([-0.0734,  0.3600,  0.0707,  0.1023,  0.1533, -0.0307,  0.0791, -0.1671])
tensor([    -0.0205,     -0.0202,     -0.0101,     -0.0220,      0.0222,
            -0.0001,      0.0195,     -0.0308])
tensor(-0.0621)


## Dot Product
- Assume we have two vectors $a$ and $b$.
  - $a = [a_1, a_2, a_3, a_4, ..., a_n]$
  - $b = [b_1, b_2, b_3, b_4, ..., b_n]$
- $a \cdot b$ = $\sum _{i=1}^n a_ib_i$  = $a_1b_1 + a_2b_2 + a_3b_3 + a_4b_4 + ... + a_nb_n$


In [83]:
# calculate P(potter|harry)

vector_of_potter = word_vectors[tok2idx['potter']]
vector_of_harry, vector_of_potter

dot_product_value_between_potter_harry = torch.dot(vector_of_harry, vector_of_potter)
dot_product_value_between_potter_harry

tensor(-0.0621)

In [ ]:
# we can get the dot product value for every other words in the vocab
# to get  P(word | harry)
word_dot_dict = {}
for word in filtered_vocab:
  word_vector = word_vectors[tok2idx[word]]
  dot_product_value = torch.dot(vector_of_harry, word_vector)
  word_dot_dict[word] = dot_product_value.item()

word_dot_dict

In [ ]:
# we have to convert our prediction into probability distribution
# P(word|harry)
# sum of [P(a|harry), ..., P(potter|harry), ... P(ron|harry), ... ] = 1

# every probability should be in range (0, 1) (greater than 0, smaller than 1)
# this can be handled by taking exponential of dot product values, divided by total sum




## Matrix Multiplication as Dot Product
![img](https://mkang32.github.io/images/python/khan_academy_matrix_product.png)

In [ ]:
word_idx, context_word_idx, len(sample_sentence)

(21, 23, 23)